In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D , UpSampling2D , Dense , Input , Flatten
from tensorflow.nn import relu , softmax
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
print("version of tensorflow is : ",tf.__version__)

In [2]:
(train_data , train_label) , (test_data , test_label) = cifar10.load_data()
print("Shape of train_data is : ",train_data.shape)
print("Shape of train_label is : ",train_label.shape)
print("Shape of test_data is : ",test_data.shape)
print("Shape of test_label is : ",test_label.shape)

Shape of train_data is :  (50000, 32, 32, 3)
Shape of train_label is :  (50000, 1)
Shape of test_data is :  (10000, 32, 32, 3)
Shape of test_label is :  (10000, 1)


In [3]:
train_label = to_categorical(train_label)
test_label  = to_categorical(test_label)

print("Shape of train_data is : ",train_data.shape)
print("Shape of train_label is : ",train_label.shape)
print("Shape of test_data is : ",test_data.shape)
print("Shape of test_label is : ",test_label.shape)

Shape of train_data is :  (50000, 32, 32, 3)
Shape of train_label is :  (50000, 10)
Shape of test_data is :  (10000, 32, 32, 3)
Shape of test_label is :  (10000, 10)


In [4]:
resnet_block = ResNet50(include_top= False , weights='imagenet' , input_shape=(224,224,3))
for layer in resnet_block.layers:
    layer.trainable = False
# resnet_block.summary()

In [5]:
def model():
    input_ = Input(shape=(32,32,3))
    x = UpSampling2D((7,7))(input_)
    x = resnet_block(x)
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(1024 , activation=relu)(x)
    x = Dense(512  , activation=relu)(x)
    x = Dense(10   , activation=softmax)(x)

    model = tf.keras.Model(inputs = input_ , outputs = x)
    model.compile(loss = tf.keras.losses.categorical_crossentropy , optimizer = 'rmsprop' , metrics = ['acc'])
    return model

In [6]:
transfer_model = model()
transfer_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 512)              

In [7]:
his = transfer_model.fit(train_data , train_label , epochs = 10 , validation_data=(test_data , test_label))

Epoch 1/10
1563/1563 [==============================] - 410s 262ms/step - loss: 0.9606 - acc: 0.6775 - val_loss: 0.9077 - val_acc: 0.7103
Epoch 2/10
1563/1563 [==============================] - 408s 261ms/step - loss: 0.7185 - acc: 0.7583 - val_loss: 0.8003 - val_acc: 0.7394
Epoch 3/10
1563/1563 [==============================] - 408s 261ms/step - loss: 0.6490 - acc: 0.7830 - val_loss: 0.7853 - val_acc: 0.7541
Epoch 4/10
1563/1563 [==============================] - 407s 261ms/step - loss: 0.6027 - acc: 0.7990 - val_loss: 0.8281 - val_acc: 0.7555
Epoch 5/10
1563/1563 [==============================] - 408s 261ms/step - loss: 0.5660 - acc: 0.8135 - val_loss: 0.7302 - val_acc: 0.7946
Epoch 6/10
1563/1563 [==============================] - 408s 261ms/step - loss: 0.5300 - acc: 0.8240 - val_loss: 0.7656 - val_acc: 0.7847
Epoch 7/10
1563/1563 [==============================] - 408s 261ms/step - loss: 0.4994 - acc: 0.8347 - val_loss: 0.7331 - val_acc: 0.7903
Epoch 8/10
1563/1563 [============

In [8]:
transfer_model.save("transfer_model.h5")